In [2]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from scrapy.selector import Selector

import re, requests, csv
import lxml

# Regex
regex_url = r"(http(s?):\/\/(www)?[A-Za-z0-9-?\/\.]+)"
regex_hastags = r"(#[A-Za-z0-9]{2,})"

# File url
file_url = '../datasets/everydayhealth.txt'
dest_url = './run_data/everyday_scrapt_v2.csv'

# Basic list
csv_header = ['tweetid','date','title']
frame_header = ['tweetid','date','title','what']

# Support scripts
%run -i scripts.py

In [2]:
# -------------------------------------------------
frame = pd.read_table(file_url, sep='|', header= None, names = csv_header, encoding="utf-8")
frame.head()

,tweetid,date,title
0,304596701757464576,Thu Feb 21 14:21:27 +0000 2013,#FastFood Makes Up 11 Percent of #Calories in ...
1,304595191329853441,Thu Feb 21 14:15:27 +0000 2013,"10 snacks to help you lose weight, burn fat, a..."
2,304587659018371072,Thu Feb 21 13:45:31 +0000 2013,10 foods that boost your skin AND slim your wa...
3,304580073380524032,Thu Feb 21 13:15:22 +0000 2013,What a heart attack feels like in women (it's ...
4,304572560270573569,Thu Feb 21 12:45:31 +0000 2013,#McDonalds oatmeal has almost 7 teaspoons of s...


In [3]:
frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3238 entries, 0 to 3237
Data columns (total 3 columns):
tweetid    3238 non-null int64
date       3238 non-null object
title      3238 non-null object
dtypes: int64(1), object(2)
memory usage: 76.0+ KB


In [4]:
# -------------------------------------------------
frame.loc[:,'title'][0]

'#FastFood Makes Up 11 Percent of #Calories in U.S. #Diet http://bit.ly/VymaIc'

In [5]:
# -------------------------------------------------
# Split Url & Text
frame['url'] = frame['title'].map(lambda row: get_url(regex_url, row))
frame['title'] = frame['title'].map(split_txt_form_url)

http://bit.ly/VymaIc
http://bit.ly/TeQPqk
http://trib.al/LDbxW6
http://bit.ly/InrWrC
http://bit.ly/GGM5bA
http://trib.al/4eRjME
http://bit.ly/yWptJ7
http://bit.ly/R7OpqA
http://bit.ly/HxUJEn
http://bit.ly/HXuSpU
http://bit.ly/yTBw80
http://bit.ly/xbkHC0
http://trib.al/wMjJUG
http://bit.ly/YLPgnc
http://trib.al/iLlHOR
http://bit.ly/wyXSKB
http://bit.ly/HIMacH
http://bit.ly/Hb4f20
http://trib.al/YXvpJx
http://bit.ly/U4HEdQ
http://trib.al/NxtEcR
http://bit.ly/T2lpU6
http://bit.ly/YnEeSn
http://trib.al/Z0JjTe
http://bit.ly/TngoWp
http://bit.ly/zutyZW
http://bit.ly/UlfGut
http://bit.ly/yCNop5
http://paper.li/EverydayHealth/1340975022
http://bit.ly/155TOrX
http://bit.ly/Vnwmkm
http://bit.ly/xrJsuW
http://ow.ly/hTS7J
http://bit.ly/11t6Hy7
http://bit.ly/NNsFUd
http://bit.ly/10dj5kk
http://bit.ly/VIs4TP
http://ow.ly/hTS2e
http://trib.al/eD95Gw
http://bit.ly/15w0Uam
http://bit.ly/155GPGE
http://bit.ly/ZUrqV8
http://bit.ly/UKV7Yh
http://bit.ly/VQCBj8
http://bit.ly/xjp08G
http://ow.ly/hTzaV
http:/

http://bit.ly/yyC84F
http://bit.ly/SHHR7E
http://trib.al/9Pldmh
http://bit.ly/Sx9PD0
http://trib.al/JnzWMb
http://paper.li/EverydayHealth/1340975022
http://trib.al/97Jc6R
http://bit.ly/zvx3JM
http://bit.ly/A4W8GE
http://trib.al/vqMqAp
http://bit.ly/IogcmA
http://bit.ly/Hjpe5x
http://bit.ly/HxUGbN
http://trib.al/6lboTD
http://trib.al/5V7BGw
http://trib.al/uwwZRm
http://trib.al/GO9Rqm
http://bit.ly/R7OMl3
http://bit.ly/yh71tu
http://bit.ly/O3CL2c
http://bit.ly/O2bFF2
http://bit.ly/zMjPF8
http://bit.ly/zZ85Ri
http://bit.ly/HusqfQ
http://bit.ly/GI0ltR
http://bit.ly/GEqleT
http://bit.ly/QdPZeS
http://bit.ly/yeE4TC
http://bit.ly/JTXnti
http://bit.ly/HplLld
http://bit.ly/GXfNqG
http://bit.ly/GH74Ef
http://bit.ly/KlbmFC
http://bit.ly/Hju0jo
http://trib.al/EFuPCQ
http://bit.ly/HjpjX2
http://bit.ly/w1ixyE
http://bit.ly/GXp5mA
http://bit.ly/Tngi0R
http://trib.al/ztJ4kl
http://trib.al/wcDMnO
http://bit.ly/H5aWUu
http://bit.ly/KRHL75
http://bit.ly/HIM84y
http://bit.ly/JpSuXz
http://trib.al/3Zgp5E
h

http://bit.ly/UE8PyW
New research finds a key to how the body stores fat: http://bit.ly/RCHbAV


http://bit.ly/RCHbAV
The top new health devices for 2013: http://bit.ly/UcdcP2


http://bit.ly/UcdcP2
The 9 worst ways to treat #depression http://bit.ly/SJyAYb


http://bit.ly/SJyAYb
Energy Drinks and Caffeine: Health Risks Updates http://bit.ly/UDU4MG


http://bit.ly/UDU4MG
Ladies, it's time for a little self-love to boost your health (seriously): http://bit.ly/JpSrLc


http://bit.ly/JpSrLc
What could be making you fat: http://bit.ly/WkPej0


http://bit.ly/WkPej0
Researchers Detect an Anti #Autism Advantage in Females http://bit.ly/XKRikA


http://bit.ly/XKRikA
Emerging #SARS Like Virus Well Suited to Attack Humans http://bit.ly/XKRhgt


http://bit.ly/XKRhgt
Early Exposure to Gluten May Help Babies Avoid #CeliacRisk http://bit.ly/Y2YaO6


http://bit.ly/Y2YaO6
How long should a cough last? You might be surprised to find out: http://bit.ly/VGpt0s


http://bit.ly/VGpt0s
Better memory, better


http://bit.ly/UqwHoi
Make this one simple swap for fast weight loss: http://trib.al/3kfv9d


http://trib.al/3kfv9d
Can you guess which employees are the most sleep deprived? http://bit.ly/z6nenQ


http://bit.ly/z6nenQ
US #cancer death rates drop 20% http://bit.ly/VbkXJ1


http://bit.ly/VbkXJ1
10 ways to squeeze in exercise into your busy day: http://bit.ly/TngiOG


http://bit.ly/TngiOG
Rekindle your #relationship with expert tips from @DrLauraBerman http://bit.ly/Z5xdvW


http://bit.ly/Z5xdvW
7 life-enhancing reasons to eat fish: http://bit.ly/SJyBuZ


http://bit.ly/SJyBuZ
How can #Giant pandas cure human infections? http://bit.ly/S5oMyg


http://bit.ly/S5oMyg
Teen drinking may boost odds of precancerous breast changes: http://bit.ly/HnpJYD


http://bit.ly/HnpJYD
Bacon-lovers, beware: http://bit.ly/x9Hxq3 #nitrates #lung


http://bit.ly/x9Hxq3
When people who have never smoked get lung cancer: http://trib.al/hbxiyh


http://trib.al/hbxiyh
Everyday Health Daily Digest is out! http://pa



RT @EinsteinMed: A1:Metformin is most commonly used diabetes med. It lowers blood sugar by helping the body use its own insulin more effectively #healthtalk


RT @JoSakimura: A6 Losing weight, exercising regularly, eating a healthy diet can help improve blood sugar control in combination w/ medication #HealthTalk


Q7: What is metformin and how is it used for diabetes prevention and treatment? #HealthTalk


RT @CodingNotes: Very illuminating chat on #diabetes. Tune in: #healthtalk.


RT @EinsteinMed: A6: ur doctor may prescribe other medicines 2 reduce complications risk, such as pills to lower cholesterol or blood pressure. #healthtalk


RT @EinsteinMed: A6: About half of people with type 2 diabetes need treatment with insulin. #healthtalk


RT @EinsteinMed: A6: Most people w/type 2 diabetes eventually need more than 1 medicine to control blood sugar. #healthtalk


RT @EinsteinMed: A6: The choice of medicine depends on many factors inc. efficacy/how well it lowers blood sugar, cost 


http://bit.ly/xvVHLc
RT @Steve_Nahorni: @EverydayHealth A1: 6 pack abs, gym 3x a week. no excuses! #healthtalk


RT @eram_h: @EverydayHealth 1. Shed few pounds 2. Eat Healthy 3. Stick with Exercise routine. #healthtalk


RT @AngelaTague: @EverydayHealth A1: Maintain my recent weight loss, tone up and continue to follow a whole foods diet at least 75% of the time. #healthtalk


Q1: What are 3 of your health/fitness goals for 2013? #healthtalk


Also, remember to include the hashtag #HealthTalk in your tweets! Let's get started!


@missssalvatore Welcome! So happy you're joining us! #HealthTalk


Welcome! RT @eram_h: @EverydayHealth I'm from India living in Netherlands. Hello!!! #HealthTalk


Just a reminder to include A1, A2, etc in your answers as they correspond to questions labeled Q1, Q2, etc. #HealthTalk


@loseit welcome! so happy you could join us today! #HealthTalk


This is @Ashley_LizWelch from behind the @EverydayHealth handle! Excited for today's #HealthTalk


Introduce you



RT @NPF: A1: Triggers are not universal. What may cause one person's #psoriasis to become active, may not affect another. #healthtalk #healthtalk


RT @NPF: About 10% of population inherits 1 or more genes that create a predisposition to #psoriasis. Only 2-3% develop it #healthtalk #healthtalk


RT @NPF: A1: To develop #psoriasis, u need combo of genes that cause it &amp; be exposed 2 external factors known as "triggers" #healthtalk #healthtalk


Q1: What triggers should you avoid that could aggravate psoriasis? #HealthTalk


And of course include the hashtag #HealthTalk in all your tweets. Let's get started!


@ssddottv Welcome! #HealthTalk


@teaminspire welcome! So happy you're here! #HealthTalk


Just a reminder to answer questions using A1, A2, etc as they correspond to questions labeled Q1, Q2, etc. #HealthTalk


RT @NPF: #Psoriasis is the most common autoimmune disease in the United States. #healthtalk. Learn about @NPF: http://www.psoriasis.org #healthtalk


http://www.psoria

RT @drdairy50: @eatsmartbd dairy protein has been shown to stimulate muscle synthesis and repair, good suggestion! #healthtalk


RT @eatsmartbd: A: A fave shake: Greek #yogurt, small bk’d sweet potato, banana, ice. Delish + gr8 4 energy! #healthtalk @everydayhealth


RT @eatsmartbd: A2: U have to know your body: some pple need to eat b4 #exercise while others pref pumping on empty! #healthtalk #menshealth


RT @DisabilityGuide: A2: Have a small snack an hour before you work out to give you energy. Afterwards, have a larger, protein packed meal! #HealthTalk


RT @eatsmartbd: A2: Shake up your pre- or post-workout with a smoothie. Skim or 1% #milk provides energizing combo of pro+carb #healthtalk


RT @johnlapuma: A1 Several studies have shown that men who eat more protein / fewer carbs for breakfast have greater satiety and eat less later #healthtalk


Q2 Is it better to eat before or after a workout? #healthtalk #menshealth #exercise


RT @eatsmartbd: A1 Skipping #breakfast meals skipp

In [6]:
frame.head()

,tweetid,date,title,url
0,304596701757464576,Thu Feb 21 14:21:27 +0000 2013,#FastFood Makes Up 11 Percent of #Calories in ...,http://bit.ly/VymaIc
1,304595191329853441,Thu Feb 21 14:15:27 +0000 2013,"10 snacks to help you lose weight, burn fat, a...",http://bit.ly/TeQPqk
2,304587659018371072,Thu Feb 21 13:45:31 +0000 2013,10 foods that boost your skin AND slim your wa...,http://trib.al/LDbxW6
3,304580073380524032,Thu Feb 21 13:15:22 +0000 2013,What a heart attack feels like in women (it's ...,http://bit.ly/InrWrC
4,304572560270573569,Thu Feb 21 12:45:31 +0000 2013,#McDonalds oatmeal has almost 7 teaspoons of s...,http://bit.ly/GGM5bA


In [7]:
frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3238 entries, 0 to 3237
Data columns (total 4 columns):
tweetid    3238 non-null int64
date       3238 non-null object
title      3238 non-null object
url        3238 non-null object
dtypes: int64(1), object(3)
memory usage: 101.3+ KB


In [8]:
# -------------------------------------------------
frame_urls = frame['url'].values

In [9]:
# -------------------------------------------------
# Run for this jupyter notebook
title_attempt = True
for key, url in enumerate(frame_urls):
    try:
        # Start the connection
        res = requests.get(url)        

        # Check return header
        if res.status_code == 200:
            print("Header:  ", 200)
            print(url)
            
            # Initialize XPath
            xpath_selector = Selector(text=res.text)
            
            list_content = dict()
            
            # Scrapt Text with XPath
            contents = xpath_selector.xpath('//div[@class="article-body"]/p/text()').extract()
            contents = join_txt(contents)
            print(len(contents))
#             # Tags
#             tags = xpath_selector.xpath('').extract()     
#             print(len(tags))
            # Author
            author = xpath_selector.xpath('//a[@class="article_link"]/span/text()').extract()
            if len(author):
                author = author[0]
            else:
                author = ''
            print(len(author))
            list_content.update({'contents': contents,'tags': '','author': author})
            
            if title_attempt:
                print("TItle: ", title_attempt)
                df_temp = pd.DataFrame(columns = list(list_content.keys()))
                frame = pd.concat([frame, df_temp])
                print(frame.columns)                
                title_attempt = False
                
            for k_, v_ in list_content.items():
                print(k_," : " ,len(v_))
                frame.loc[key,k_] = v_
            print("\n")

        else:
            print("['ERRORS'] {} , at {}".format(res.status_code, url))
    except:
        pass
    

Header:   200
http://bit.ly/VymaIc
2605
32
TItle:  True
Index(['author', 'contents', 'date', 'tags', 'title', 'tweetid', 'url'], dtype='object')
contents  :  2605
tags  :  0
author  :  32




scripts.py:38: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  # Find all possible hastags


Header:   200
http://bit.ly/TeQPqk
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://trib.al/LDbxW6
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://bit.ly/InrWrC
1457
24
contents  :  1457
tags  :  0
author  :  24


Header:   200
http://bit.ly/GGM5bA
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://trib.al/4eRjME
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://bit.ly/yWptJ7
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://bit.ly/R7OpqA
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://bit.ly/HxUJEn
3928
36
contents  :  3928
tags  :  0
author  :  36


Header:   200
http://bit.ly/HXuSpU
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://bit.ly/yTBw80
1968
24
contents  :  1968
tags  :  0
author  :  24


Header:   200
http://bit.ly/xbkHC0
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://trib.al/wMjJUG
3487
18
contents  :  3487
tags  :  0
author  :  18


Heade

Header:   200
http://ow.ly/hRjQ0
1520
12
contents  :  1520
tags  :  0
author  :  12


Header:   200
http://bit.ly/OeONa1
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://bit.ly/H3VdoC
1814
0
contents  :  1814
tags  :  0
author  :  0


Header:   200
http://trib.al/t0EGSq
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://bit.ly/WOfW3A
3052
13
contents  :  3052
tags  :  0
author  :  13


Header:   200
http://bit.ly/Ajmo1n
2793
13
contents  :  2793
tags  :  0
author  :  13


Header:   200
http://bit.ly/wsIEBi
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://trib.al/qomllw
825
21
contents  :  825
tags  :  0
author  :  21


Header:   200
http://bit.ly/H4JvKw
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://ow.ly/hRjQ0
1520
12
contents  :  1520
tags  :  0
author  :  12


Header:   200
http://trib.al/ATQowW
3255
31
contents  :  3255
tags  :  0
author  :  31


Header:   200
http://bit.ly/GEqlLV
3312
15
contents  :  3312
tags  : 

Header:   200
http://trib.al/vyzEmK
2734
20
contents  :  2734
tags  :  0
author  :  20


Header:   200
http://bit.ly/HQDm1Y
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://bit.ly/GXRJE4
1097
22
contents  :  1097
tags  :  0
author  :  22


Header:   200
http://trib.al/6eGLzO
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://trib.al/wapOSu
1114
12
contents  :  1114
tags  :  0
author  :  12


Header:   200
http://bit.ly/SHHSsc
4727
16
contents  :  4727
tags  :  0
author  :  16


Header:   200
http://bit.ly/11LrQnt
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://trib.al/pAOVwa
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://bit.ly/Sx9Nv1
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://bit.ly/R7OYAJ
1560
24
contents  :  1560
tags  :  0
author  :  24


Header:   200
http://bit.ly/OeN2tv
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://bit.ly/FOt7Kk
6378
18
contents  :  6378
tags  :  0
aut

...

Header:   200
http://bit.ly/wAzxSK
4710
12
contents  :  4710
tags  :  0
author  :  12


Header:   200
http://bit.ly/ITK5bS
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://bit.ly/yNwdcv
2579
20
contents  :  2579
tags  :  0
author  :  20


Header:   200
http://trib.al/2Jt8uC
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://trib.al/7uq3GL
1530
13
contents  :  1530
tags  :  0
author  :  13


Header:   200
http://bit.ly/13Eaw0J
2513
30
contents  :  2513
tags  :  0
author  :  30


Header:   200
http://bit.ly/13DVUiX
1760
11
contents  :  1760
tags  :  0
author  :  11


Header:   200
http://ow.ly/gQqcD
556
12
contents  :  556
tags  :  0
author  :  12


Header:   200
http://bit.ly/FPN9oP
3014
30
contents  :  3014
tags  :  0
author  :  30


Header:   200
http://bit.ly/GCnjYp
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://bit.ly/W73Cgc
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://trib.al/G15l0r
1579
13
contents  :  1579
ta

Header:   200
http://trib.al/iIuJTY
1541
30
contents  :  1541
tags  :  0
author  :  30


Header:   200
http://trib.al/26Rpea
2469
13
contents  :  2469
tags  :  0
author  :  13


Header:   200
http://trib.al/sH6I8h
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://trib.al/1DTqGK
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://bit.ly/InrSrS
2560
38
contents  :  2560
tags  :  0
author  :  38


Header:   200
http://bit.ly/SHHTMR
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://bit.ly/zmowu0
1740
24
contents  :  1740
tags  :  0
author  :  24


Header:   200
http://bit.ly/ITTtNf
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://paper.li/EverydayHealth/1340975022
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://bit.ly/SJyNuq
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://bit.ly/GEqmiT
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://bit.ly/zl3XjZ
1809
22
contents  :  1809
ta

Header:   200
http://bit.ly/HZdinW
3518
15
contents  :  3518
tags  :  0
author  :  15


Header:   200
http://trib.al/y6E385
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://trib.al/y6E385
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://bit.ly/HxUKs2
2483
30
contents  :  2483
tags  :  0
author  :  30


Header:   200
http://bit.ly/HxUKs2
2483
30
contents  :  2483
tags  :  0
author  :  30


Header:   200
http://bit.ly/A3yExG
5072
24
contents  :  5072
tags  :  0
author  :  24


Header:   200
http://bit.ly/AqxHsi
3878
32
contents  :  3878
tags  :  0
author  :  32


Header:   200
http://bit.ly/AqxHsi
3878
32
contents  :  3878
tags  :  0
author  :  32


Header:   200
http://bit.ly/zlQgWP
6190
12
contents  :  6190
tags  :  0
author  :  12


Header:   200
http://bit.ly/SHHRET
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://bit.ly/TkpOSr
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://bit.ly/TkpOSr
0
0
contents  :  0
tags  : 

Header:   200
http://bit.ly/A7BLqe
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://bit.ly/HIM5FX
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://bit.ly/Aegeto
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://bit.ly/HIY9a9
4016
15
contents  :  4016
tags  :  0
author  :  15


Header:   200
http://trib.al/IHNdjO
1742
13
contents  :  1742
tags  :  0
author  :  13


Header:   200
http://trib.al/SOPQPd
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://trib.al/AvKJ4R
3322
22
contents  :  3322
tags  :  0
author  :  22


Header:   200
http://trib.al/XSa2nb
1583
16
contents  :  1583
tags  :  0
author  :  16


Header:   200
http://bit.ly/HHE9Rz
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://trib.al/3kfv9d
1833
13
contents  :  1833
tags  :  0
author  :  13


Header:   200
http://trib.al/wTfQcI
9029
17
contents  :  9029
tags  :  0
author  :  17


Header:   200
http://bit.ly/z6nenQ
4631
27
contents  :  4631
tags 

Header:   200
http://bit.ly/wlkV4d
2088
30
contents  :  2088
tags  :  0
author  :  30


Header:   200
http://bit.ly/Hb4bz7
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://bit.ly/VV9UhU
1641
30
contents  :  1641
tags  :  0
author  :  30


Header:   200
http://bit.ly/11jyNuk
4229
27
contents  :  4229
tags  :  0
author  :  27


['ERRORS'] 404 , at http://bit.ly/wBjror
Header:   200
http://trib.al/CpW7FF
4526
18
contents  :  4526
tags  :  0
author  :  18


Header:   200
http://bit.ly/VLEtrc
0
11
contents  :  0
tags  :  0
author  :  11


Header:   200
http://trib.al/BrHlkE
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://trib.al/WNQs2r
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://bit.ly/OeOCMa
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://trib.al/oeoOb9
3458
22
contents  :  3458
tags  :  0
author  :  22


Header:   200
http://bit.ly/11j5ZCa
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://bit.ly/HT3

In [10]:
# -------------------------------------------------
frame.head()

,author,contents,date,tags,title,tweetid,url
0,"Steven Reinberg, HealthDay News",". from restaurants such as McDonald's, Burge...",Thu Feb 21 14:21:27 +0000 2013,,#FastFood Makes Up 11 Percent of #Calories in ...,3.045967e+17,http://bit.ly/VymaIc
1,,,Thu Feb 21 14:15:27 +0000 2013,,"10 snacks to help you lose weight, burn fat, a...",3.045952e+17,http://bit.ly/TeQPqk
2,,,Thu Feb 21 13:45:31 +0000 2013,,10 foods that boost your skin AND slim your wa...,3.045877e+17,http://trib.al/LDbxW6
3,Jaimie Dalessio Clayton,— What does a . feel like?. Before you answe...,Thu Feb 21 13:15:22 +0000 2013,,What a heart attack feels like in women (it's ...,3.045801e+17,http://bit.ly/InrWrC
4,,,Thu Feb 21 12:45:31 +0000 2013,,#McDonalds oatmeal has almost 7 teaspoons of s...,3.045726e+17,http://bit.ly/GGM5bA


In [11]:
frame.to_csv(dest_url, sep='|', encoding='utf-8',index=False)

In [3]:
# If End of jupyter-notebook, reset the "frame" Dataframe as 4 columns
frame = pd.read_csv(file_url, sep='|', header= None, names = csv_header, encoding="utf-8")
frame_content = pd.read_csv(dest_url, sep='|', encoding="utf-8")
### Writing content into text file
### ================================
file_names = frame['tweetid'].values

for key, name in enumerate(file_names):
    
    # Construct the file name
    located_file = './training/' + str(name) + '.txt'
#     success_date = './date/success_date.txt'
    reg_special = r"([^a-zA-Z\s]+)"
    
    try:
    
        # Writing on file
        contents = str(frame_content['contents'].iloc[key])
        print(key, ' : ' ,len(contents))
        if len(contents) > 10:
            # Open file for writing
            f= open(located_file,"w+")
            
            # Clean content a bit
            contents = re.sub(reg_special, '', contents)
            
            # Start writing content to file.
            f.write(contents)
            
            # close file connection
            f.close()
            
            #Write success load date
#             f1 = open(success_date, "a")
#             f1.write(frame['date'].iloc[key])
#             f1.close()
        else:
            print("[Out] key: ", key) 
    except:
        pass

### ================================

0  :  2605
1  :  3
[Out] key:  1
2  :  3
[Out] key:  2
3  :  1457
4  :  3
[Out] key:  4
5  :  3
[Out] key:  5
6  :  3
[Out] key:  6
7  :  3
[Out] key:  7
8  :  3928
9  :  3
[Out] key:  9
10  :  1968
11  :  3
[Out] key:  11
12  :  3487
13  :  3
[Out] key:  13
14  :  1133
15  :  3357
16  :  3
[Out] key:  16
17  :  3
[Out] key:  17
18  :  3
[Out] key:  18
19  :  3741
20  :  2626
21  :  3
[Out] key:  21
22  :  4283
23  :  3
[Out] key:  23
24  :  3405
25  :  3
[Out] key:  25
26  :  4470
27  :  3
[Out] key:  27
28  :  3
[Out] key:  28
29  :  1630
30  :  2897
31  :  3
[Out] key:  31
32  :  1383
33  :  994
34  :  3
[Out] key:  34
35  :  2950
36  :  2184
37  :  1520
38  :  3
[Out] key:  38
39  :  4035
40  :  3
[Out] key:  40
41  :  797
42  :  2461
43  :  3
[Out] key:  43
44  :  3
[Out] key:  44
45  :  1383
46  :  2933
47  :  3
[Out] key:  47
48  :  3
[Out] key:  48
49  :  3
[Out] key:  49
50  :  3
[Out] key:  50
51  :  3
[Out] key:  51
52  :  3
[Out] key:  52
53  :  3
[Out] key:  53
54  :  3
[O

495  :  3
[Out] key:  495
496  :  3
[Out] key:  496
497  :  3
[Out] key:  497
498  :  3
[Out] key:  498
499  :  3
[Out] key:  499
500  :  3
[Out] key:  500
501  :  3
[Out] key:  501
502  :  3
[Out] key:  502
503  :  3
[Out] key:  503
504  :  3
[Out] key:  504
505  :  3
[Out] key:  505
506  :  3
[Out] key:  506
507  :  3
[Out] key:  507
508  :  3
[Out] key:  508
509  :  3
[Out] key:  509
510  :  3
[Out] key:  510
511  :  3
[Out] key:  511
512  :  3
[Out] key:  512
513  :  3
[Out] key:  513
514  :  3
[Out] key:  514
515  :  3
[Out] key:  515
516  :  3
[Out] key:  516
517  :  3
[Out] key:  517
518  :  3
[Out] key:  518
519  :  3
[Out] key:  519
520  :  3
[Out] key:  520
521  :  3
[Out] key:  521
522  :  3
[Out] key:  522
523  :  3
[Out] key:  523
524  :  3
[Out] key:  524
525  :  6583
526  :  3
[Out] key:  526
527  :  3
[Out] key:  527
528  :  3
[Out] key:  528
529  :  3
[Out] key:  529
530  :  3
[Out] key:  530
531  :  3
[Out] key:  531
532  :  3
[Out] key:  532
533  :  3
[Out] key:  533

1042  :  3100
1043  :  3
[Out] key:  1043
1044  :  3
[Out] key:  1044
1045  :  3
[Out] key:  1045
1046  :  3326
1047  :  3
[Out] key:  1047
1048  :  1378
1049  :  2446
1050  :  3
[Out] key:  1050
1051  :  2151
1052  :  2994
1053  :  2138
1054  :  3536
1055  :  2064
1056  :  3
[Out] key:  1056
1057  :  502
1058  :  3
[Out] key:  1058
1059  :  4098
1060  :  1760
1061  :  3
[Out] key:  1061
1062  :  2661
1063  :  3183
1064  :  3
[Out] key:  1064
1065  :  3
[Out] key:  1065
1066  :  3
[Out] key:  1066
1067  :  3
[Out] key:  1067
1068  :  3
[Out] key:  1068
1069  :  3
[Out] key:  1069
1070  :  3
[Out] key:  1070
1071  :  3
[Out] key:  1071
1072  :  3
[Out] key:  1072
1073  :  3
[Out] key:  1073
1074  :  3
[Out] key:  1074
1075  :  3
[Out] key:  1075
1076  :  3
[Out] key:  1076
1077  :  3
[Out] key:  1077
1078  :  3
[Out] key:  1078
1079  :  3
[Out] key:  1079
1080  :  3
[Out] key:  1080
1081  :  3
[Out] key:  1081
1082  :  3
[Out] key:  1082
1083  :  3
[Out] key:  1083
1084  :  3
[Out] key:

1537  :  4060
1538  :  2005
1539  :  3
[Out] key:  1539
1540  :  3
[Out] key:  1540
1541  :  4206
1542  :  2544
1543  :  1551
1544  :  3
[Out] key:  1544
1545  :  3
[Out] key:  1545
1546  :  3
[Out] key:  1546
1547  :  3
[Out] key:  1547
1548  :  3
[Out] key:  1548
1549  :  3
[Out] key:  1549
1550  :  1074
1551  :  3
[Out] key:  1551
1552  :  3
[Out] key:  1552
1553  :  1114
1554  :  3
[Out] key:  1554
1555  :  3
[Out] key:  1555
1556  :  2115
1557  :  6190
1558  :  3
[Out] key:  1558
1559  :  3
[Out] key:  1559
1560  :  3566
1561  :  4858
1562  :  1539
1563  :  2687
1564  :  3
[Out] key:  1564
1565  :  3
[Out] key:  1565
1566  :  2727
1567  :  3255
1568  :  3
[Out] key:  1568
1569  :  3
[Out] key:  1569
1570  :  2653
1571  :  3
[Out] key:  1571
1572  :  5453
1573  :  2469
1574  :  4325
1575  :  3
[Out] key:  1575
1576  :  3017
1577  :  3
[Out] key:  1577
1578  :  3511
1579  :  3
[Out] key:  1579
1580  :  1579
1581  :  3
[Out] key:  1581
1582  :  3
[Out] key:  1582
1583  :  3
[Out] key

1999  :  8326
2000  :  5418
2001  :  1545
2002  :  3
[Out] key:  2002
2003  :  3
[Out] key:  2003
2004  :  4305
2005  :  3
[Out] key:  2005
2006  :  3531
2007  :  4339
2008  :  2149
2009  :  6043
2010  :  4156
2011  :  3
[Out] key:  2011
2012  :  3
[Out] key:  2012
2013  :  3054
2014  :  4883
2015  :  3
[Out] key:  2015
2016  :  4537
2017  :  7642
2018  :  1941
2019  :  3312
2020  :  1802
2021  :  1853
2022  :  3
[Out] key:  2022
2023  :  3
[Out] key:  2023
2024  :  3
[Out] key:  2024
2025  :  3
[Out] key:  2025
2026  :  1445
2027  :  699
2028  :  3
[Out] key:  2028
2029  :  3
[Out] key:  2029
2030  :  3
[Out] key:  2030
2031  :  2909
2032  :  3
[Out] key:  2032
2033  :  3
[Out] key:  2033
2034  :  3
[Out] key:  2034
2035  :  1847
2036  :  8724
2037  :  1683
2038  :  3448
2039  :  1998
2040  :  1683
2041  :  3
[Out] key:  2041
2042  :  1110
2043  :  2193
2044  :  7191
2045  :  2821
2046  :  3
[Out] key:  2046
2047  :  2639
2048  :  3
[Out] key:  2048
2049  :  4308
2050  :  3
[Out] key:

2483  :  1683
2484  :  4526
2485  :  3267
2486  :  2138
2487  :  3469
2488  :  2617
2489  :  3
[Out] key:  2489
2490  :  3
[Out] key:  2490
2491  :  4137
2492  :  2981
2493  :  3
[Out] key:  2493
2494  :  3
[Out] key:  2494
2495  :  918
2496  :  2511
2497  :  3
[Out] key:  2497
2498  :  3
[Out] key:  2498
2499  :  3959
2500  :  540
2501  :  3
[Out] key:  2501
2502  :  3
[Out] key:  2502
2503  :  4261
2504  :  3487
2505  :  3
[Out] key:  2505
2506  :  3
[Out] key:  2506
2507  :  4088
2508  :  2486
2509  :  3
[Out] key:  2509
2510  :  3
[Out] key:  2510
2511  :  3890
2512  :  3
[Out] key:  2512
2513  :  3
[Out] key:  2513
2514  :  4478
2515  :  3
[Out] key:  2515
2516  :  3
[Out] key:  2516
2517  :  1579
2518  :  3
[Out] key:  2518
2519  :  3
[Out] key:  2519
2520  :  3
[Out] key:  2520
2521  :  3
[Out] key:  2521
2522  :  3
[Out] key:  2522
2523  :  2501
2524  :  2942
2525  :  1870
2526  :  3
[Out] key:  2526
2527  :  3
[Out] key:  2527
2528  :  2626
2529  :  4095
2530  :  2042
2531  : 

2972  :  737
2973  :  3
[Out] key:  2973
2974  :  2890
2975  :  3
[Out] key:  2975
2976  :  2793
2977  :  3
[Out] key:  2977
2978  :  2692
2979  :  3
[Out] key:  2979
2980  :  1901
2981  :  3
[Out] key:  2981
2982  :  3
[Out] key:  2982
2983  :  6671
2984  :  2665
2985  :  3
[Out] key:  2985
2986  :  3
[Out] key:  2986
2987  :  3
[Out] key:  2987
2988  :  3
[Out] key:  2988
2989  :  3
[Out] key:  2989
2990  :  1683
2991  :  3
[Out] key:  2991
2992  :  4954
2993  :  1998
2994  :  1998
2995  :  1538
2996  :  1538
2997  :  3
[Out] key:  2997
2998  :  3
[Out] key:  2998
2999  :  1891
3000  :  1891
3001  :  1312
3002  :  1312
3003  :  3
[Out] key:  3003
3004  :  3
[Out] key:  3004
3005  :  3433
3006  :  3433
3007  :  3
[Out] key:  3007
3008  :  3
[Out] key:  3008
3009  :  3
[Out] key:  3009
3010  :  3
[Out] key:  3010
3011  :  3
[Out] key:  3011
3012  :  2282
3013  :  2282
3014  :  4526
3015  :  4526
3016  :  2193
3017  :  2193
3018  :  3
[Out] key:  3018
3019  :  3
[Out] key:  3019
3020  :